<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Загрузка данных клиентов 

In [2]:
df_insurance = pd.read_csv('/datasets/insurance.csv')

Обзор дата фрейма 

In [3]:
df_insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, все признаки имеют числовой тип

Обзор таблицы с признаками

In [4]:
df_insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Корреляция признаков по методу Спирмена 

In [5]:
df_insurance.drop('Пол', axis = 1).corr(method = 'spearman')

,Возраст,Зарплата,Члены семьи,Страховые выплаты
Возраст,1.000000,-0.020238,-0.001503,0.548589
Зарплата,-0.020238,1.000000,-0.025492,-0.005549
Члены семьи,-0.001503,-0.025492,1.000000,-0.029210
Страховые выплаты,0.548589,-0.005549,-0.029210,1.000000


Заметен средний уровень положительной корреляции страховых выплат с возрастом клиента

Сравнение группы клиентов, которая имела страховые случаи с теми, у кого  они отсутствовали

In [6]:
df_insurance_case = df_insurance[df_insurance['Страховые выплаты'] > 0]
df_not_insurance_case = df_insurance[df_insurance['Страховые выплаты'] == 0]

In [7]:
df_insurance_case['Возраст'].describe()

count    564.000000
mean      46.542553
std        3.888312
min       42.000000
25%       44.000000
50%       46.000000
75%       49.000000
max       65.000000
Name: Возраст, dtype: float64

In [8]:
df_not_insurance_case['Возраст'].describe()

count    4436.000000
mean       28.970694
std         6.599344
min        18.000000
25%        23.000000
50%        29.000000
75%        34.000000
max        42.000000
Name: Возраст, dtype: float64

Видим что в группе где присутствуют страховые случаи показатели возраста выше. Это говорит о возрастозависимости страховых случаев.
Возраст  - самый важный признак (из тех что присутствуют), характеризующий риск наступления страхового случая

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Формула нахождения весов для преобразованной матрицы:**

$$
w' = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T X P)^{-1} P^T X^T y 
$$

**Предсказания для преобразованной  матрицы:**

$$
a' = XPw' =XP (P^T X^T X P)^{-1} P^T X^T y = \\
    = XPP^{-1} (X^T X )^{-1} P^{T-1}P^T X^T y =\\
    = XE (X^T X )^{-1}E X^T y = \\
    = X (X^T X )^{-1} X^T y = \\
    = X w =>\\
    a' = X w = a
$$

**Ответ:** 

При умножении признаков  на обратимую матрицу, качество предсказаний линейной регрессии не измениться.


**Обоснование:** 

Формула расчета предсказаний при преобразовании остается идентичной: 
$$
a' = Xw = a
$$


## Алгоритм преобразования

**Алгоритм**
* Выделение обучающих и целевых  признаков 
* Подсчет количества признаков в обучающей выборке 
* Получение случайной квадратной обратимой матрицы с размерностью равной признакам в обучающей  матрицы 
* Преобразование матрицы признаков путем ее умножения на полученную случайную матрицу 


**Обоснование**

При умножении на обратимую матрицы получим матрицу с неинтерпретируемыми результатами  без знания обратимой матрицы 

## Проверка алгоритма

Получение обучающих и целевых  признаков

In [9]:
features = df_insurance.drop('Страховые выплаты', axis=1).values
target = df_insurance['Страховые выплаты'].values

Функция для преобразования матрицы признаков

In [10]:
def X_modified(matrix_X,seed):
    dimension = features.shape[1]
    
    try:
        state = np.random.RandomState(seed=seed)
        matrix_P = state.normal(size = (dimension,dimension))
        inv_P =  np.linalg.inv(matrix_P)
        return matrix_X @ inv_P,inv_P
    except np.linalg.LinAlgError:
        X_modified(matrix_X,seed)
   

Сплитирование

In [11]:
# защита данных
features_modified,inv_P = X_modified(features,12345)

features_train_raw,features_test_raw,target_train_raw,target_test_raw = train_test_split(features,target,
                                                                                   test_size = .25, random_state = 12345)


features_train_new,features_test_new,target_train_new,target_test_new = train_test_split(features_modified,target,
                                                                                   test_size = .25, random_state = 12345)


Восстановление данных по известной обратной матрице

In [12]:
features_decoder = features_modified @ np.linalg.inv(inv_P)
pd.DataFrame(features_decoder,
             columns = df_insurance.columns[:-1]) \
            .round(1).astype('int').head(5)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


In [13]:
df_insurance.head(5).astype('int')

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


**Проверка качества модели** 

In [14]:
model = LinearRegression()
model.fit(features_train_raw, target_train_raw)
predictions = model.predict(features_test_raw)
print('Метрика R2 на исходных признаках:',r2_score(target_test_raw, predictions))


model.fit(features_train_new, target_train_new)
predictions = model.predict(features_test_new)
print('Метрика R2 на преобразованных признаках:',r2_score(target_test_new, predictions))

Метрика R2 на исходных признаках: 0.43522757127026546
Метрика R2 на преобразованных признаках: 0.435227571270297


Вывод:

   При предсказании методом линейной регрессии, подтверждена  возможность защиты данных методом умножения на обратимую матрицу